In [1]:
# Importing essential libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
num_epochs = 5
batch_size = 32
learning_rate = 0.001

# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a uniform size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='/kaggle/input/inaturalist2/inaturalist_12K/train', transform=transform)

# Split dataset into training and validation sets
training_data_size = int(0.8 * len(dataset))
validation_data_size = len(dataset) - training_data_size
training_dataset, validation_dataset = random_split(dataset, [training_data_size, validation_data_size])

# Create data loaders
training_data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

testing_dataset = ImageFolder(root='/kaggle/input/inaturalist2/inaturalist_12K/val', transform=transform)

testing_data_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=True)

#classes = dataset.classes
#print(classes)

In [2]:
# Defining function to freeze the layers
def freeze_layers(model, freeze_percent=0.5):
    total_layers = len(list(model.parameters()))
    num_layers_to_freeze = int(total_layers * freeze_percent)
    layers_frozen = 0

    for name, param in model.named_parameters():
        if layers_frozen < num_layers_to_freeze:
            param.requires_grad = False
            layers_frozen += 1
        else:
            break
'''
# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

# Verify if layers are frozen

for name, param in model.named_parameters():
    print(name, param.requires_grad)
    
print(model)'''

'\n# Load pre-trained ResNet50 model\nmodel = models.resnet50(pretrained=True)\nnum_features = model.fc.in_features\nmodel.fc = nn.Linear(num_features, 10)\n\n# Verify if layers are frozen\n\nfor name, param in model.named_parameters():\n    print(name, param.requires_grad)\n    \nprint(model)'

In [3]:
!pip install wandb  # Installing wandb

In [4]:
# Importing wandb and essential libraries

import wandb
import numpy as np
from types import SimpleNamespace
import random

In [5]:
wandb.login(key='cd7a6c2259e8886dc269bbf6f0f9e55089d3beeb')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
#defining the sweep configuration
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep test(check)',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'freezing' : {
            'values' : [0.7,0.8,0.9]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='Deep_Learning_Assignment2_PartB')

Create sweep with ID: vg8vf67x
Sweep URL: https://wandb.ai/prabhat-kumar/Deep_Learning_Assignment2_PartB/sweeps/vg8vf67x


In [7]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init(entity = 'prabhat-kumar') as run:

        run_name="-frz"+str(wandb.config.freezing)
        wandb.run.name=run_name
#         actv = get_activation(wandb.config.activation)
        
        # Load pre-trained ResNet50 model
        model = models.resnet50(pretrained=True)
        
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)
        model = model.to(device)
        
        freeze_layers(model, freeze_percent=wandb.config.freezing)
        
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)   # Using Adam as an optimizer

        # Training loop
        for epoch in range(num_epochs):
            #initializing the necessary values
            total_training_data = 0
            correct_predicted_train_data = 0
            loss1 = 0
            # Train the model
            model.train()
            for images, labels in training_data_loader:
                # Moving images and labels to GPU
                images = images.to(device)
                labels = labels.to(device)
        
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
        
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss1 = loss1 + loss.item()
                
                # Calculate the training accuracy
                max_value, predicted_by_model = torch.max(outputs, 1)
                total_training_data = total_training_data + labels.size(0)
                correct_predicted_train_data = correct_predicted_train_data + (predicted_by_model == labels).sum().item()
                
            train_accuracy = 100*correct_predicted_train_data/total_training_data
            print(f'Epoch number {epoch+1}, Training Accuracy: {train_accuracy:.2f}%') # Printing training accuracy
            wandb.log({'train_loss': round(loss1 / len(training_data_loader), 2)})
            wandb.log({'train_accuracy': round(train_accuracy, 2)})

            #wandb.log({'train_loss':running_loss/len(train_loader)})
            #wandb.log({'train_accuracy':train_accuracy})
            
        
    
            # Validate the model
            model.eval()
            with torch.no_grad():
                # Initializing the values
                correct_predicted_validation_data = 0
                total_validation_data = 0
                for images, labels in validation_data_loader:
                    
                    images = images.to(device)  # Moving images to GPU
                    labels = labels.to(device)  # Moving labels to GPU
            
                    # Forward pass
                    outputs = model(images)
                    max_value, predicted_by_model = torch.max(outputs, 1)
            
                    # Compute accuracy
                    total_validation_data = total_validation_data + labels.size(0)
                    correct_predicted_validation_data = correct_predicted_validation_data + (predicted_by_model == labels).sum().item()
            
        
                val_accuracy = 100 * correct_predicted_validation_data / total_validation_data
                print(f'Epoch number {epoch+1}, Validation Accuracy: {val_accuracy:.2f}%')
                wandb.log({'val_accuracy':val_accuracy})
                wandb.log({'epoch':epoch+1})
    

        print('Finished Training') # Here training finished

wandb.agent(sweep_id, function=main,count=1) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: 37gl0833 with config:
wandb: 	freezing: 0.7
wandb: Currently logged in as: prabhat-kumar. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 149MB/s] 


Epoch number 1, Training Accuracy: 57.37%
Epoch number 1, Validation Accuracy: 61.45%
Epoch number 2, Training Accuracy: 72.91%
Epoch number 2, Validation Accuracy: 66.85%
Epoch number 3, Training Accuracy: 81.70%
Epoch number 3, Validation Accuracy: 69.95%
Epoch number 4, Training Accuracy: 88.21%
Epoch number 4, Validation Accuracy: 67.25%
Epoch number 5, Training Accuracy: 91.59%
Epoch number 5, Validation Accuracy: 70.15%
Finished Training


epoch,▁▃▅▆█
train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
val_accuracy,▁▅█▆█
epoch,5
train_accuracy,91.59
train_loss,0.26
val_accuracy,70.15
